# Workspace for Project Euler problems

## Problem 12

The sequence of triangle numbers is generated by adding the natural numbers. So the 7th triangle number would be $1 + 2 + 3 + 4 + 5 + 6 + 7 = 28$. The first ten terms would be:

$$1, 3, 6, 10, 15, 21, 28, 36, 45, 55, ...$$

Let us list the factors of the first seven triangle numbers:
<pre><code>
 1: 1
 3: 1,3
 6: 1,2,3,6
10: 1,2,5,10
15: 1,3,5,15
21: 1,3,7,21
28: 1,2,4,7,14,28
</code></pre>

We can see that 28 is the first triangle number to have over five divisors.

What is the value of the first triangle number to have over five hundred divisors?

In [1]:
import primetools as pt
import numbers, math
import tools.misc as misc
import logging, sys
from functools import reduce

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

log = logging.getLogger("main")

# DEBUG, INFO, WARNING, ERROR, CRITICAL
log.setLevel(logging.INFO)

### Approach

Given a prime factorization of a number $N$:

$$ N = p_1^{x_1}p_2^{x_2}...p_n^{x_n}$$

The number of multiples is simply the number of unique prime factorizations that can be made from a subset of that factorization. The number of such factorizations can be thought of as, for each unique prime $p_i$, choosing the number of instances between 0 and $x_i$, of which there are of course $x_i + 1$ choices. Therefore, the number of combinations is simply:

$$ (x_1+1)(x_2+1)...(x_n+1) $$

The prime factorization of 500 is $2^{2}5^{3}$. This means a number with three different prime factors each occurring four times and two occurring once will have exactly 500 factors. As an example:


In [2]:
(2**4) * (3**4) * (5**4) * 7 * 11

62370000

More examples

In [3]:
posmin=2**4 * 3**4 * 5**2 * 7 * 11 * 13
posmin2=2**4 * 3**2 * 5**2 * 7**2 * 11 * 13
posmin3=2**6 * 3**3 * 5**2 * 7**2 * 11
posmin4=2**7 * 3**3 * 5**3 * 7**2
posmin5=2**7 * 3**4 * 5**2 * 7 * 11

def checkPosMin(num):
    log.info(f"******\nCandidate for lowest number with 500 factors: {num}")
    factors=pt.factor(num)
    log.info("Factorization of %d: %s", num, factors)

    divisors=pt.getFactors(num)

    log.info("Number of divisors for %d: %d", num, len(divisors))
    log.debug(divisors)
    
checkPosMin(posmin)
checkPosMin(posmin2)
checkPosMin(posmin3)
checkPosMin(posmin4)
checkPosMin(posmin5)

INFO:main:******
Candidate for lowest number with 500 factors: 32432400
INFO:main:Factorization of 32432400: {2: 4, 3: 4, 5: 2, 7: 1, 11: 1, 13: 1}
INFO:main:Number of divisors for 32432400: 600
INFO:main:******
Candidate for lowest number with 500 factors: 25225200
INFO:main:Factorization of 25225200: {2: 4, 3: 2, 5: 2, 7: 2, 11: 1, 13: 1}
INFO:main:Number of divisors for 25225200: 540
INFO:main:******
Candidate for lowest number with 500 factors: 23284800
INFO:main:Factorization of 23284800: {2: 6, 3: 3, 5: 2, 7: 2, 11: 1}
INFO:main:Number of divisors for 23284800: 504
INFO:main:******
Candidate for lowest number with 500 factors: 21168000
INFO:main:Factorization of 21168000: {2: 7, 3: 3, 5: 3, 7: 2}
INFO:main:Number of divisors for 21168000: 384
INFO:main:******
Candidate for lowest number with 500 factors: 19958400
INFO:main:Factorization of 19958400: {2: 7, 3: 4, 5: 2, 7: 1, 11: 1}
INFO:main:Number of divisors for 19958400: 480


This may be bad, but I believe this is the smallest number with more than 500 factors. I believe this because if you look at the factors of the last number, 23284800, then you see that there is no way to take away prime factors from one place and add them to another to produce a lower number with the same number of divisors. Right now the product is $7\times4\times3\times3\times2$. Reducing any of those numbers would drop the product below 500. So to remove any factor, one must be added somewhere else, but that cannot be done without raising the overall number.

So for example it is tempting to remove 11 as a factor and reduce the overall number by a lot, but since there is only one 11 factor, removing it cuts the total number of divisors in half, and those divisors must be made up somehow by adding smaller factors with a product less than 11, and there is no way to double the number of divisors while multiplying the final product by less than 11. Adding more existing factors provides diminishing returns in terms of contributing to the total number of divisors.

In [4]:
mymin=23284800
closeroot=math.sqrt(mymin*2)
print(closeroot)
closeroot=math.floor(closeroot)
closetri=closeroot*(closeroot+1)/2
print(f"closest triangle number: {closetri}. The number in the sequence that produced it: {closeroot}")

6824.1922598942065
closest triangle number: 23286900.0. The number in the sequence that produced it: 6824


So what this implies is that we only have to start checking triangle numbers that are greater than 23286900. And since a triangle number has the formula

$$\frac{n(n+1)}{2}$$

Then to find the closest triangle number greater than 23286900, we just take the square root of $2\times23286900$ as an estimate for $n$, round up, and apply the formula

In [5]:
def countDivisors(factors):
    """Given a prime factorization, count the number of divisors."""
    return reduce(lambda x,y:x*y, (x+1 for x in factors.values()))

In [6]:
def checktris(start, primes=None):
    """Check triangle numbers in sequence to see if each one has """
    tri=start*(start+1)/2
    for i in range(start, 100000):
        if primes is None:
            f=pt.factor(tri)
        else:
            f=pt.factorFaster(tri, primes)
        numofdivs=countDivisors(f)
        if numofdivs > 500:
            print (f"Found it: {tri}")
            break
        tri += i

# This returns in reasonable time
# checktris(6824)
# This does not
# checktris(2)

In [10]:

maybeit=76576500
assert maybeit == 12375 * 12376 / 2
itfacs=pt.factor(maybeit)
print(f"Root: {math.sqrt(maybeit*2)}")
print(f"Maybe it: {maybeit}. Factors for maybe it: {itfacs}. Number of divisors: {countDivisors(itfacs)}")


Root: 12375.499989899397
Maybe it: 76576500. Factors for maybe it: {2: 2, 3: 2, 5: 3, 7: 1, 11: 1, 13: 1, 17: 1}. Number of divisors: 576
